# Setup

In [1]:
import numpy as np
from scipy import optimize

# Objective function

In [2]:
def simulate(sigma_psi,sigma_xi,T,psi,xi):
    
    # a. time loop
    p = np.zeros((T*12,N))
    y = np.zeros((T*12,N))
    for t in range(12*T):
        if t == 0:
            p[t,:] = psi[t,:]
        else:
            p[t,:] = p[t-1,:] + psi[t,:]
        y[t,:] = p[t,:] + xi[t,:]
    
    # b. aggregate to annual
    Y = np.zeros((T,N))
    for t in range(T):
        Y[t,:] = np.log(np.sum(np.exp(y[12*t:12*(t+1),:]),axis=0)) 
    
    return Y

In [3]:
def cov_psi(Y):
    return np.cov(Y[2,:]-Y[1,:],Y[3,:]-Y[0,:])[0,1]

def cov_xi(Y):
    return -np.cov(Y[2,:]-Y[1,:],Y[1,:]-Y[0,:])[0,1]

In [4]:
def obj(x,T,psi_raw,xi_raw):

    # a. unpack
    sigma_psi = x[0]
    sigma_xi = x[1]
    
    # b. scale shocks
    psi = -0.5*sigma_psi**2 + sigma_psi*psi_raw
    xi = -0.5*sigma_xi**2 + sigma_xi*xi_raw
    
    # c. simulate
    Y = simulate(sigma_psi,sigma_xi,T,psi,xi)
    
    # d. objective function
    obj = 0
    obj += 1000*(0.0054 - cov_psi(Y))**2
    obj += 1000*(0.0072 - cov_xi(Y))**2
    
    return obj

# Optimize

**Settings:**

In [5]:
N = 1_000_000
T = 4

np.random.seed(1066)
psi_raw = np.random.normal(size=(T*12,N))
xi_raw = np.random.normal(size=(T*12,N))

**Call:**

In [6]:
x0 = np.zeros(2)
x0[0] = np.sqrt(0.0054/12)
x0[1] = np.sqrt(0.0072*12)

result = optimize.minimize(obj,x0,args=(T,psi_raw,xi_raw),options={'disp':True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 10


**Estimates:**

In [7]:
sigma_psi = result.x[0]
sigma_xi = result.x[1]

print(f'sigma_psi = {sigma_psi:.8f} [sigma_psi^2 = {sigma_psi**2:.8f}]')
print(f'sigma_xi  = {sigma_xi:.8f} [sigma_xi^2  = {sigma_xi**2:.8f}]')

sigma_psi = 0.02122033 [sigma_psi^2 = 0.00045030]
sigma_xi  = 0.30467480 [sigma_xi^2  = 0.09282673]


**Implied covariances:**

In [8]:
psi = -0.5*sigma_psi**2 + sigma_psi*psi_raw
xi = -0.5*sigma_xi**2 + sigma_xi*xi_raw
Y = simulate(sigma_psi,sigma_xi,T,psi,xi)

In [9]:
print(f'cov_psi: {cov_psi(Y):.8f}')
print(f'cov_xi: {cov_xi(Y):.8f}')

cov_psi: 0.00540000
cov_xi: 0.00720000
